# Introduction to LLM API Usage

This notebook demonstrates how to use OpenAI's API to interact with Large Language Models (LLMs), specifically GPT-4o models. It provides a structured approach for building chat applications with both text and image processing capabilities.



Load the Openai api key, which is set in .env

In [1]:
import os
from dotenv import load_dotenv

In [3]:
# Load environment variables from .env file
load_dotenv()

# Get the OpenAI API key
openai_api_key = os.getenv('OPENAI_API_KEY')

Try single-round chat with gpt-4o

In [4]:
from openai import OpenAI
client = OpenAI(api_key=openai_api_key)

In [5]:
completion = client.chat.completions.create( model="gpt-4o",
messages=[
{"role": "system", "content": "You are a helpfull assistant。"}, {"role": "user", "content": "what is 3 + 4?"}
] )
print(completion.choices[0].message.content)

3 + 4 equals 7.


multi-turn chatbot

In [21]:
import re
from IPython.display import display, Code, Markdown

def extract_url_and_text(input_text):
    """
    Extract URL and descriptive text from user input.

    Parameters:
    input_text (str): User input text, which may contain a URL and descriptive text.

    Returns:
    tuple: Contains descriptive text and extracted URL, if no URL is found, returns (input_text, None)
    """

    # Use regular expression to detect URL
    url_pattern = re.compile(r'https?://[^\s]+')
    url_match = url_pattern.search(input_text)

    if url_match:
        # Extract URL and remove it from the original text
        url = url_match.group(0)
        description = input_text.replace(url, '').strip()  # Remove URL and clean up spaces
        return description, url
    else:
        return input_text, None

In [22]:
def create_message(role, content):
    """
    Create a message with a specified role and content.

    Parameters:
    role (str): Role of the message sender (e.g., "user", "assistant").
    content (str): Message content.

    Returns:
    dict: Message with role and content.
    """
    return {"role": role, "content": content}

In [23]:
def create_user_message_with_image(text, url):
    """
    Create a user message containing an image URL.

    Parameters:
    text (str): User message text.
    url (str): Image URL.

    Returns:
    dict: User message containing text and image URL.
    """
    return {
        "role": "user",
        "content": [
            {"type": "text", "text": text},
            {"type": "image_url", "image_url": {"url": url}}
        ]
    }

In [24]:
import os
import base64

def encode_image_to_base64(image_path):
    """
    Encode a local image to a base64 string.

    Parameters:
    image_path (str): Local image path.

    Returns:
    str: Base64 encoded string.
    """
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')


def create_user_message_with_local_image(text, image_path):
    """
    Create a user message containing a local image.

    Parameters:
    text (str): User message text.
    image_path (str): Local image path.

    Returns:
    dict: User message containing text and image.
    """
    base64_image = encode_image_to_base64(image_path)
    return {
        "role": "user",
        "content": [
            {"type": "text", "text": text},
            {
                "type": "image_url",
                "image_url": {
                    "url": f"data:image/png;base64,{base64_image}"
                }
            }
        ]
    }



In [25]:
def process_user_input(input_text):
    """
    Determine whether the user input contains a URL or local PNG image, and generate the corresponding message format.

    Parameters:
    input_text (str): The user's input text.

    Returns:
    dict: The generated user message containing text or image URL.
    """
    
    description, url = extract_url_and_text(input_text)
    
    # Check if it's a local PNG file
    is_local_png = False
    local_png_path = ""
    
    # If no URL found, check for text ending with .png, which might be a local file path
    if not url and ".png" in input_text.lower():
        # Try to extract potential PNG file paths
        potential_paths = [word.strip() for word in input_text.split() 
                          if word.strip().lower().endswith('.png')]
        
        for path in potential_paths:
            if os.path.isfile(path):
                is_local_png = True
                local_png_path = path
                # Remove file path from input text, use the rest as description
                description = input_text.replace(path, "").strip()
                break

    if is_local_png:
        # If it's a local PNG file, generate a message with the local image
        if not description:
            description = "Please analyze the content of this image."  # Use default prompt if no description provided
        return create_user_message_with_local_image(description, local_png_path)
    elif url:
        # If a URL is detected, generate a message with the image link
        if not description:
            description = "Please analyze the content of this image."  # Use default prompt if no description provided
        return create_user_message_with_image(description, url)
    else:
        # Otherwise generate a normal text message
        return create_message("user", description)

In [26]:
def chat_with_gpt4o(client, messages):
    """
    Core function for multi-round conversation using the GPT-4o model.

    Parameters:
    client (OpenAI): An instantiated OpenAI client.
    messages (list): A list of messages containing conversation context, including user, assistant, and system messages.

    Returns:
    str: The reply content generated by the GPT-4o model.
    """

    response = client.chat.completions.create(
        model="gpt-4o-mini",  # Using the GPT-4o-mini model
        messages=messages
    )

    # Extract and return the reply content generated by the assistant
    return response.choices[0].message.content

In [27]:
def multi_round_chat():
    """
    Multi-round chatbot example function that demonstrates how to capture user input
    and process it based on whether it contains a URL.
    """

    # Initialize the message list, including system message
    messages = []

    # Create system message, setting the context for the conversation
    system_message = create_message("system", "You are a helpful assistant.")
    messages.append(system_message)

    while True:
        # Capture user input
        user_input = input("User: ")

        # Process user input and generate the corresponding message
        user_message = process_user_input(user_input)
        messages.append(user_message)

        # Call the GPT-4o model for a response
        assistant_reply = chat_with_gpt4o(client, messages)
        display(Markdown(f"Assistant: {assistant_reply}"))

        # Add the assistant's reply to the message list
        messages.append(create_message("assistant", assistant_reply))

        # Provide an exit mechanism, user can type 'exit' to end the conversation
        if user_input.lower() == 'exit':
            print("Conversation ended.")
            break


if __name__ == "__main__":
    # Run the multi-round conversation example
    multi_round_chat()

Assistant: The image depicts a serene landscape featuring a lake surrounded by mountains. In the foreground, there are several rocks and two small trees growing on a larger rock formation in the water. The sky is dramatic with dark clouds and lightning, suggesting an impending storm. Reflections of the rocks and trees can be seen in the calm water of the lake, and there are hints of light from structures in the background amidst the lush greenery.

Assistant: The coloring in the image features a mix of muted and natural tones. The sky displays dark shades of gray and hints of gold or orange from the setting sun or storm light. The mountains are depicted in deep greens and grays, while the water has a reflective quality, showcasing shades of green and gray. The rocks appear in various natural stone colors, such as gray and brown, complementing the overall earthy palette. The overall effect conveys a moody and dramatic atmosphere.

Assistant: The mood of the image is dramatic and somewhat tense due to the presence of dark, stormy clouds and lightning. The serene lake and natural elements provide a contrast to the impending storm, creating a sense of anticipation or foreboding. Overall, the combination of these elements evokes a feeling of awe and wonder, as well as a hint of tranquility amidst the turbulent atmosphere.

Assistant: If you have any more questions or need assistance in the future, feel free to ask. Have a great day!

Conversation ended.
